# Mini-Project II
The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area. The project consists of following tasks:

#### Instructions:

- pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area using [these instructions](https://www.yelp.com/developers/documentation/v3/business_search)). If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts, not the actual number of places we were able to get.
- create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've used and created sqlite3 databases before in the activity [**SQL in Python**](https://data.compass.lighthouselabs.ca/b9e08cd5-68c6-490c-a32b-a66f01bf53e1).
- compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.
- choose the top 10 POIs based on the popularity (number of reviews or average rating) ([Yelp](https://www.yelp.com/developers/documentation/v3/business), [Foursquare](https://developer.foursquare.com/docs/api-reference/venues/details/)).
- (Stretch) By implementing [travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem), how much time would it take to visit all of these places? ([Directions API](https://developers.google.com/maps/documentation/directions/start) from google will be helpful here). We will have to find travel time between all places (top 10). We can use [ortools](https://developers.google.com/optimization/routing/tsp) from Google to effectively implement TSP. These tools are very powerful and [easy to install](https://developers.google.com/optimization/install).

In [1]:
# Build data base of restaurants, bars and various points of Interest (POIs)
# Find out which API has better coverage in the selected area. 
#- Toronto - Yorkville (lat_long: 43.6702287372475, -79.39126856350059)

#'39.06003891,-86.49843283'

In [2]:
# Import libraries
import requests
import pandas as pd
import sqlite3
from sqlite3 import Error
import foursquare
from config import fsq, yep
from IPython.display import JSON
from numpy.random import default_rng

In [ ]:
# 1. Pull data about various POIs in the area through API - yelp and foursquare
# 2. Create SQLite database and store the data bout the POIs. - design structure of the database
# 3. Compare result using SQL or Pandas (it's up to you) and see which API has a better coverage of the area.
# 4. Choose the top 10 POIs based on the popularity

## Pull data about various POIs in the area through API - yelp and foursquare

Foursquare:

* 950 Regular Calls/Day
* 50 Premium Calls/Day
* 1 Photo per Venue
* 1 Tip per Venue

Yelp:
* Daily API limit 5000
* Remaining API Calls 5000

#### Foursquare

In [3]:
radius = '1000'

In [4]:
# Foursquare connection
foursquare_id = fsq['client_id']
foursquare_secret = fsq['client_secret']
client = foursquare.Foursquare(client_id=foursquare_id, client_secret=foursquare_secret)

#Select 30 - for details

In [5]:
# Get business Id
## How many venues?
#soup,museums,coffeshops,musicvenues '4bf58dd8d48988d1dd931735,4bf58dd8d48988d181941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d1e5931735'
def get_foursquare_venues(lat_long, catId, radius):
    '''
    Returns a list of information for "Nightlife Spot" in a radius of 5km based on lat and long provided
     Category id is for "Nightlife Spot"
    '''
    venues  = client.venues.search(params={'categoryId' : catId, 'll': lat_long, 'radius': radius, 'limit': '50'})

    return venues

In [6]:
def get_foursquare_venue_details(venue_id):
    
    venue_details = client.venues(venue_id)
    
    return venue_details

In [7]:
# Get Foursquare venues
foursquare_venues = get_foursquare_venues('43.6702287372475, -79.39126856350059', '4bf58dd8d48988d1dd931735,4bf58dd8d48988d181941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d1e5931735', radius)

In [8]:
JSON(foursquare_venues)

<IPython.core.display.JSON object>

In [9]:
# Create dataFrame and save it as csv
foe_df = pd.DataFrame(foursquare_venues['venues'])

In [10]:
foe_df.to_csv('./csv/foursquare_venues.csv')

In [11]:
# Get Foursquare venue details for one venue to analyze data
foursquare_venue_details = get_foursquare_venue_details('56046c7d498e66c67551d0e3')

In [12]:
JSON(foursquare_venue_details)

<IPython.core.display.JSON object>

#### Yelp

In [17]:
# Yelp connection
yelp_id = yep['client_id']
yelp_api_key = yep['api_key']
headers = {'Authorization': 'Bearer %s' % yelp_api_key}
root_endpoint = 'https://api.yelp.com/v3'

In [18]:
# Get Yelp Venues
def get_yelp_venues(latitude, longitude, categories, radius):
    api_method = '/businesses/search'
    endpoint = root_endpoint + api_method
    response = requests.get(endpoint, 
                      params={'categories': categories, 'latitude': latitude, 'longitude': longitude, 'radius': radius, 'limit': '50'},
                      headers=headers)
    
    status_code, result = response.status_code, response.json()
    
    
    print(f'Request status code: {status_code}')
    
    return result

In [19]:
# Get Yelp Venues details
def get_yelp_venue_details(venue_id):
    api_method = '/businesses/'
    endpoint = root_endpoint + api_method + venue_id

    response = requests.get(endpoint, headers=headers)
    
    status_code, result = response.status_code, response.json()
    
    
    print(f'Request status code: {status_code}')
    
    return result

In [20]:
# Get Venues
yelp_venues = get_yelp_venues('43.6702287372475','-79.39126856350059', 'soup,museums,coffee,musicvenues', radius)

Request status code: 200


In [21]:
JSON(yelp_venues) # 86

<IPython.core.display.JSON object>

In [22]:
# Create dataFrame and save it as csv
yep_df = pd.DataFrame(yelp_venues['businesses'])

In [23]:
yep_df.to_csv('yelp_venues.csv')

In [24]:
# Get Details to analyze data
venue_id = 'WKcVegcUjAld1S737a_QHw'
#venue_id = "g3t2AQQVT2tWD64Huu7xVA"
venue_details = get_yelp_venue_details(venue_id)

Request status code: 200


In [25]:
JSON(venue_details)

<IPython.core.display.JSON object>

## Get Sample venues

In [29]:
# Sample venues
# due to limited API calls we will get 30 - Foursquare
fq_rng = default_rng()
f_30 = fq_rng.choice(48, size=30, replace=False).tolist()
f_30.sort()

In [31]:
# Sample venues
# due to limited API calls we will get 30
yp_rng = default_rng()
yep_sample = yp_rng.choice(48, size=30, replace=False)
y_30 = yep_sample.tolist()
y_30.sort()

In [37]:
# get Foursquare venues sample - 30
#sample_foursquare = 
sample_foe = foe_df.iloc[f_30]
sample_foe.to_csv('csv/sample_foursquare_venues.csv')

In [38]:
# get Yelp venues sample - 30
#sample_yelp = 
sample_yep = yep_df.iloc[y_30]
sample_yep.to_csv('csv/sample_yelp_venues.csv')

In [ ]:
# Foursquare details
all_foe_details = []
for index, value in sample_foe['id'].items():
    f_venue_details_temp = get_foursquare_venue_details(value)
    all_foe_details.append(f_venue_details_temp['venue'])

In [41]:
foursquare_venue_details_df = pd.DataFrame(all_foe_details)
foursquare_venue_details_df.to_csv('csv/foursquare_venue_details.csv')

In [ ]:
# Yelp details
all_yep_details = []
for index, value in sample_yep['id'].items():
    y_venue_details_temp = get_yelp_venue_details(value)
    all_yep_details.append(y_venue_details_temp)

In [47]:
yelp_venue_details_df = pd.DataFrame(all_yep_details)
yelp_venue_details_df.to_csv('csv/yelp_venue_details.csv')

## Compare fields 

In [49]:
foursquare_venue_details_df.columns

Index(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories',
       'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating',
       'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere',
       'specials', 'photos', 'reasons', 'description', 'page', 'hereNow',
       'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours',
       'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox',
       'attributes', 'bestPhoto', 'colors', 'parent', 'hierarchy', 'venuePage',
       'storeId'],
      dtype='object')

In [50]:
yelp_venue_details_df.columns

Index(['id', 'alias', 'name', 'image_url', 'is_claimed', 'is_closed', 'url',
       'phone', 'display_phone', 'review_count', 'categories', 'rating',
       'location', 'coordinates', 'photos', 'price', 'hours', 'transactions',
       'special_hours', 'messaging'],
      dtype='object')

In [51]:
foursquare_venue_details_df.dtypes

id                   object
name                 object
contact              object
location             object
canonicalUrl         object
categories           object
verified               bool
stats                object
url                  object
price                object
likes                object
dislike                bool
ok                     bool
rating              float64
ratingColor          object
ratingSignals       float64
allowMenuUrlEdit       bool
beenHere             object
specials             object
photos               object
reasons              object
description          object
page                 object
hereNow              object
createdAt             int64
tips                 object
shortUrl             object
timeZone             object
listed               object
hours                object
popular              object
seasonalHours        object
defaultHours         object
pageUpdates          object
inbox                object
attributes          

In [52]:
yelp_venue_details_df.dtypes

id                object
alias             object
name              object
image_url         object
is_claimed          bool
is_closed           bool
url               object
phone             object
display_phone     object
review_count       int64
categories        object
rating           float64
location          object
coordinates       object
photos            object
price             object
hours             object
transactions      object
special_hours     object
messaging         object
dtype: object

# Mini-Project II SQLite
create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've used and created sqlite3 databases before in the activity SQL in Python.

In [55]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
        
    return connection

In [56]:
connection = create_connection("db/poi.sqlite")

connection to SQLite DB successful


In [57]:
# Function to execute queries to create tables, instert records and so on.
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [58]:
# Function to execute queries to read tables
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [59]:
create_venues_table = """
CREATE TABLE IF NOT EXISTS venues_1 (
  pKey INTEGER PRIMARY KEY AUTOINCREMENT,
  id TEXT NOT NULL,
  name TEXT NOT NULL,
  category TEXT,
  app_url TEXT,
  rating REAL,
  reviews INTEGER,
  likes INTEGER,
  address TEXT, 
  crossStreet TEXT, 
  latitude REAL,
  longitude REAL,
  city TEXT,
  state TEXT,
  country TEXT,
  postalCode TEXT,
  source TEXT NOT NULL
);
"""
execute_query(connection, create_venues_table)

Query executed successfully


In [60]:
# prepare foursquare data to be inserted in the database
f_columns=['id', 'name', 'categories', 'canonicalUrl', 'rating', 'stats', 'likes']
f_test =foursquare_venue_details_df.loc[:, f_columns]

In [61]:
#project_themes['name'] = project_themes['code'].apply(lambda x: theme_dict[x])
f_test['category'] = f_test['categories'].apply(lambda x: x[0]['name'])
f_test['reviews'] = f_test['stats'].apply(lambda x: x['tipCount'])
f_test['likes'] = f_test['likes'].apply(lambda x: x['count'])

In [64]:
def insert_forsquare_data(sql):
    cursor = connection.cursor()
    try:
        for row in f_test.iterrows():
            cursor.execute("INSERT INTO venues(id, name, category, app_url, rating, reviews, likes, source) VALUES(?,?,?,?,?,?,?,?)", (row[1]['id'], row[1]['name'], row[1]['category'], row[1]['canonicalUrl'], row[1]['rating'], row[1]['reviews'] ,row[1]['likes'], 'Foursquare'))
#print(row[1]['id'], row[1]['name'], row[1]['category'], row[1]['canonicalUrl'], row[1]['rating'], row[1]['reviews'] ,row[1]['likes']) 
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
insert_forsquare_data()

In [66]:
y_columns=['id', 'name', 'categories', 'url', 'rating', 'review_count']
y_insert =yelp_venue_details_df.loc[:, y_columns]

In [67]:
filter=y_insert['id'].notna()
y_insert = y_insert[filter]

In [69]:
# take category
y_insert['category'] = y_insert['categories'].apply(lambda x: x[0]['title'])

In [70]:
def insert_yelp_data():
    cursor = connection.cursor()
    try:
        for row in y_insert.iterrows():
            cursor.execute("INSERT INTO venues(id, name, category, app_url, rating, reviews, source) VALUES(?,?,?,?,?,?,?)", (row[1]['id'], row[1]['name'], row[1]['category'], row[1]['url'], row[1]['rating'], row[1]['review_count'], 'Yelp'))
#print(row[1]['id'], row[1]['name'], row[1]['category'], row[1]['canonicalUrl'], row[1]['rating'], row[1]['reviews'] ,row[1]['likes']) 
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [71]:
insert_yelp_data()

Query executed successfully
